In [ ]:
!pip install --upgrade datasets

import json
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import ast
import nltk
import re

from collections import defaultdict
from nltk.corpus import stopwords
from huggingface_hub import login
from datasets import Dataset, DatasetDict, load_dataset
from tqdm import tqdm
from itertools import combinations

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

HF_L = "xxx"
login(token=HF_L)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
dataset = load_dataset("Ramitha/unique-records-selected-integrated-gradients")
df = pd.DataFrame(dataset['rawcases'])

README.md: 0.00B [00:00, ?B/s]

data/rawcases-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Generating rawcases split:   0%|          | 0/54 [00:00<?, ? examples/s]

In [ ]:
token_cut_off_ratio = 0.33

In [ ]:
def top_k_words(token_probs, original_text):
    if isinstance(token_probs, str):
        import ast
        token_probs = ast.literal_eval(token_probs)
    word_probs = defaultdict(float)
    current_word = ''
    current_sum = 0.0
    for pair in token_probs:
        if not isinstance(pair, (list, tuple)) or len(pair) != 2:
            continue
        token, prob = pair
        cleaned = re.sub(r'[^a-zA-Z0-9]', '', token)
        if not cleaned:
            continue
        if token.startswith('\u2581') or token.startswith('▁') or token.startswith('\u0120'):
            if current_word:
                word_probs[current_word.lower()] += current_sum
            current_word = cleaned
            current_sum = prob
        else:
            current_word += cleaned
            current_sum += prob
    if current_word:
        word_probs[current_word.lower()] += current_sum
    text_words = set(w.lower() for w in re.findall(r'\w+', original_text))
    filtered_probs = {w: p for w, p in word_probs.items() if w in text_words and w not in stop_words}
    sorted_words = sorted(filtered_probs.items(), key=lambda x: x[1], reverse=True)
    return sorted_words[:(int(len(sorted_words) * token_cut_off_ratio) + 1)]

def intersection_score(*lists):
    lists = [[w if not isinstance(w, tuple) else w[0] for w in lst] for lst in lists if lst]
    if len(lists) < 2:
        return 0.0
    sets = [set(lst) for lst in lists]
    common = set.intersection(*sets)
    union = set.union(*sets)
    denom = len(union)
    return len(common) / denom if denom else 0.0

def unique_word_count(original_text):
    stop_words = set(stopwords.words("english"))
    words = re.findall(r"\w+", original_text.lower())
    filtered_words = [w for w in words if w not in stop_words]
    filtered_words = words
    unique_words = set(filtered_words)
    return len(unique_words)

RAW RESULTS

In [ ]:
model_cols = ['question_raw_ig_tokens_llama',
              'question_raw_ig_tokens_falcon',
              'question_raw_ig_tokens_gemma',
              'question_raw_ig_tokens_mistral']

for idx, row in tqdm(df.iterrows(), total=len(df)):
    df.at[idx, 'question_iaa_all'] = intersection_score(
        *[top_k_words(row[col], row['question']) for col in model_cols]
    )
    for col1, col2 in combinations(model_cols, 2):
        col_name = f'question_iaa_{col1.split("_")[-1]}_{col2.split("_")[-1]}'
        df.at[idx, col_name] = intersection_score(
            top_k_words(row[col1], row['question']),
            top_k_words(row[col2], row['question'])
        )
    df.at[idx, 'answer_iaa_all'] = intersection_score(
        *[top_k_words(row[col], row['answer']) for col in model_cols]
    )
    for col1, col2 in combinations(model_cols, 2):
        col_name = f'answer_iaa_{col1.split("_")[-1]}_{col2.split("_")[-1]}'
        df.at[idx, col_name] = intersection_score(
            top_k_words(row[col1], row['answer']),
            top_k_words(row[col2], row['answer'])
        )
    df.at[idx, 'question_unique_words'] = unique_word_count(row['question'])
    df.at[idx, 'answer_unique_words'] = unique_word_count(row['answer'])

100%|██████████| 54/54 [00:00<00:00, 69.98it/s]


In [ ]:
average_cols = [c for c in df.columns if c.startswith('question_iaa_') or c.startswith('question_unique')]
dataset_averages = df.groupby('dataset')[average_cols].mean().reset_index()
dataset_averages

,dataset,question_iaa_all,question_iaa_llama_falcon,question_iaa_llama_gemma,question_iaa_llama_mistral,question_iaa_falcon_gemma,question_iaa_falcon_mistral,question_iaa_gemma_mistral,question_unique_words
0,alqa,0.079762,0.300866,0.276527,0.513624,0.375132,0.365681,0.320443,21.722222
1,newsqa,0.240741,0.462963,0.500000,0.555556,0.685185,0.481481,0.555556,6.722222
2,sl,0.085780,0.314242,0.341799,0.448148,0.284127,0.349852,0.260434,22.500000


In [ ]:
average_cols = [c for c in df.columns if c.startswith('answer_iaa_') or c.startswith('answer_unique')]
dataset_averages = df.groupby('dataset')[average_cols].mean().reset_index()
dataset_averages

,dataset,answer_iaa_all,answer_iaa_llama_falcon,answer_iaa_llama_gemma,answer_iaa_llama_mistral,answer_iaa_falcon_gemma,answer_iaa_falcon_mistral,answer_iaa_gemma_mistral,answer_unique_words
0,alqa,0.194511,0.400000,0.365873,0.556878,0.494974,0.487566,0.432011,55.166667
1,newsqa,0.277778,0.462963,0.462963,0.537037,0.768519,0.462963,0.537037,34.277778
2,sl,0.095062,0.367019,0.358730,0.393519,0.274603,0.288123,0.454481,51.666667


In [ ]:
hf_dataset = DatasetDict({
    'rawcases': Dataset.from_pandas(df)
})
hf_dataset.push_to_hub("Ramitha/unique-records-selected-integrated-gradients")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########| 1.25MB / 1.25MB            

CommitInfo(commit_url='https://huggingface.co/datasets/Ramitha/unique-records-selected-integrated-gradients/commit/abe1da1ffb6c430463ddc0d2c4098aae3eb5b32c', commit_message='Upload dataset', commit_description='', oid='abe1da1ffb6c430463ddc0d2c4098aae3eb5b32c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Ramitha/unique-records-selected-integrated-gradients', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Ramitha/unique-records-selected-integrated-gradients'), pr_revision=None, pr_num=None)

RELATIVE RESAULTS

In [ ]:
model_cols = ['question_ig_tokens_llama',
              'question_ig_tokens_falcon',
              'question_ig_tokens_gemma',
              'question_ig_tokens_mistral']

for idx, row in tqdm(df.iterrows(), total=len(df)):
    df.at[idx, 'question_iaa_all'] = intersection_score(
        *[top_k_words(row[col], row['question']) for col in model_cols]
    )
    for col1, col2 in combinations(model_cols, 2):
        col_name = f'question_iaa_{col1.split("_")[-1]}_{col2.split("_")[-1]}'
        df.at[idx, col_name] = intersection_score(
            top_k_words(row[col1], row['question']),
            top_k_words(row[col2], row['question'])
        )
    df.at[idx, 'answer_iaa_all'] = intersection_score(
        *[top_k_words(row[col], row['answer']) for col in model_cols]
    )
    for col1, col2 in combinations(model_cols, 2):
        col_name = f'answer_iaa_{col1.split("_")[-1]}_{col2.split("_")[-1]}'
        df.at[idx, col_name] = intersection_score(
            top_k_words(row[col1], row['answer']),
            top_k_words(row[col2], row['answer'])
        )
    df.at[idx, 'question_unique_words'] = unique_word_count(row['question'])
    df.at[idx, 'answer_unique_words'] = unique_word_count(row['answer'])


100%|██████████| 54/54 [00:00<00:00, 64.94it/s]


In [ ]:
average_cols = [c for c in df.columns if c.startswith('question_iaa_') or c.startswith('question_unique')]
dataset_averages = df.groupby('dataset')[average_cols].mean().reset_index()
dataset_averages

,dataset,question_iaa_all,question_iaa_llama_falcon,question_iaa_llama_gemma,question_iaa_llama_mistral,question_iaa_falcon_gemma,question_iaa_falcon_mistral,question_iaa_gemma_mistral,question_unique_words
0,alqa,0.050595,0.361183,0.300529,0.432876,0.152686,0.432011,0.272751,21.722222
1,newsqa,0.129630,0.370370,0.462963,0.351852,0.388889,0.518519,0.462963,6.722222
2,sl,0.046296,0.272354,0.293519,0.302690,0.282628,0.223617,0.308245,22.500000


In [ ]:
average_cols = [c for c in df.columns if c.startswith('answer_iaa_') or c.startswith('answer_unique')]
dataset_averages = df.groupby('dataset')[average_cols].mean().reset_index()
dataset_averages

,dataset,answer_iaa_all,answer_iaa_llama_falcon,answer_iaa_llama_gemma,answer_iaa_llama_mistral,answer_iaa_falcon_gemma,answer_iaa_falcon_mistral,answer_iaa_gemma_mistral,answer_unique_words
0,alqa,0.058929,0.371429,0.342593,0.442593,0.116691,0.462963,0.293939,55.166667
1,newsqa,0.236111,0.425926,0.601852,0.388889,0.361111,0.574074,0.527778,34.277778
2,sl,0.068122,0.326455,0.325926,0.353924,0.350220,0.222162,0.334568,51.666667
